In [25]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from TwitterAPI import TwitterAPI
import json
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
data.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [53]:
data['location'].drop_duplicates()

0               Afghanistan
560                  Africa
1131                Albania
1690                Algeria
2249                Andorra
                ...        
112133    Wallis and Futuna
112287                World
112880                Yemen
113394               Zambia
113931             Zimbabwe
Name: location, Length: 233, dtype: object

In [4]:
mask = data['location'] == 'Poland'
data_pl = data[mask].copy()

In [5]:
data_pl.loc[:,'date'] = pd.to_datetime(data_pl.loc[:,'date'])

In [6]:
data_pl.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       't

In [51]:
data_pl.loc[82696+300, 'total_vaccinations']


6300.0

In [72]:
data_pl.loc[:,'new_deaths']

82696     NaN
82697     NaN
82698     NaN
82699     NaN
82700     NaN
         ... 
83242    13.0
83243     9.0
83244     5.0
83245     7.0
83246     0.0
Name: new_deaths, Length: 551, dtype: float64

In [7]:
year_td = pd.Timedelta(days = 365 if pd.Timestamp.today().year % 4 != 0 else 366)

In [8]:
today = pd.Timestamp.today().normalize() - pd.Timedelta(days = 1)
today

Timestamp('2021-09-05 00:00:00')

In [9]:
today_yearago = (today - year_td).normalize()

In [12]:
mask1 = data_pl['date'] == today_yearago
mask2 = data_pl['date'] == today
numerical_features = ['total_cases','new_cases','total_deaths','new_deaths']
all_today_data = data_pl.loc[mask2,numerical_features].values[0] - data_pl.loc[mask1,numerical_features].values[0]
all_today_data

array([ 2.820097e+06, -2.440000e+02,  7.326600e+04, -1.300000e+01])

### Warning: 52 missing data in 'hosp_patients', be ware to make a flag to check if the data are available.

In [14]:
len(data_pl['hosp_patients'])

551

In [31]:
api = TwitterAPI(consumer_key = 'yRIFzDkyXao8Afy3duaEVYAsn', 
                 consumer_secret = '3ckWBRoWyZPwXl7qAprCx6piWTaa8AH6ZhmPyBfbd8bq5l17Rp', 
                 access_token_key = '1434947484401479686-E5buYUfgzFa5NXKuJOPF9Fe0kzzSUV', 
                 access_token_secret = 'VP6uZ9xgZIM24KBGiGzZA2ygOOvYNnbgBkpq0UMg5i0kG')

In [32]:
r = api.request('statuses/show/:%d' % 1432051779450163202)

In [33]:
json.loads(r.text).keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])

In [34]:
r = api.request('statuses/update', {'status':'dupa1'})

In [35]:
r.status_code

200

In [78]:
arr1 = np.array([np.nan,2])
arr2 = np.array([1,2])
np.isnan(arr1 - arr2).sum()

1

In [85]:
try:
    x = '2' - 3
except TypeError:
    x = 'dupa'

print(x)

dupa
